# Using GenePattern with Pandas

This is a quick tutorial on how to use [GenePattern](http://genepattern-notebook.org/) in conjunction with [pandas](https://pandas.pydata.org/), a popular Python library used in bioinformatics and more widely in data science. 

This tutorial assumes that you are familiar with the basics of both GenePattern and Python programming. If this is not the case, we recommend first starting with an [overview of GenePattern](http://genepattern-notebook.org/tutorials/) and going through [a Python tutorial](https://github.com/rajathkmp/Python-Lectures) before coming back to this notebook. A tutorial on how to use the [GenePattern Python library](https://github.com/genepattern/example-notebooks/blob/master/GenePattern%20Python%20Tutorial.ipynb), including how to prorammatically call GenePattern from Python, is also available and would make for useful background reading.

## Import the Libraries

The first thing you need to do is to import the Python libraries that we will be using in this tutorial. This includes the GenePattern Python library and the pandas data analysis library.

If you do not have these libraries installed, you will need to install them. This can be accomplished from the command line, using the PIP package manager. Alternatively, both libraries are also available through the conda package manager.

##### PIP (PyPI)
> pip install genepattern-python pandas

*or*

##### Conda
> conda install pandas<br/>
> conda install -c genepattern 

In [ ]:
import gp, gp.data
import pandas

## Loading a GenePattern File

We will next load a GenePattern file. For simplicity's sake, in this tutorial we are going to do this by loading the file directly from its URL. However, another common way to load a file would be to use the GenePattern Python library to connect to a GenePattern server and retrieve a file which is the result of a GenePattern analysis. We cover how to load files that way in the [GenePattern Python tutorial](https://github.com/genepattern/example-notebooks/blob/master/GenePattern%20Python%20Tutorial.ipynb). Additionally, the GenePattern Notebook extension provides a user interface for loading files from a GenePattern analysis.

The file we are going to load is a GCT file. This format contains a tab-separated matrix of gene expression values, with the columns representing the samples and the rows representing the genes. GCT files also include a few header rows at the top, which specify the format and size of the matrix. You can read more about the format and other GenePattern formats [here](http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide).

In [ ]:
gpfile = gp.GPFile(None, uri='https://datasets.genepattern.org/data/all_aml/all_aml_test.gct')

*Side Note:* This particular file contains gene expression data from ALL and AML turmor samples. This is microarray data, which has fallen out of favor somewhat in the bioinformatic community. Nevertheless, this type of dataset is small and quick to process, making it ideal for demonstration purposes in this tutorial.

## Introducing Pandas

Pandas is an popular Python library used in data exploration and analysis. The [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/) describes it as follows:

> pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python.

### The DataFrame Type

At the heart of the pandas library is an new class called a *DataFrame*. This class can be used to encapsulate, query or display any sort of labeled tabular data - basically any matrix, including column and row labels. This is the exact sort of data that you just loaded in the GenePattern file above.

*Side Note:* Pandas also provides an additional data type called a *Series*. This data type is used to represent labeled single-dimension arrays. We won't be covering it in this tutorial, as it is less relevant to most GenePattern applications, but it bears mentioning. For more on the Series data type, we recommend [this tutorial](https://github.com/jakevdp/PythonDataScienceHandbook/blob/058289a6a8a544d2bc144ba0226d6ef27986e4e0/notebooks/03.01-Introducing-Pandas-Objects.ipynb).

### Creating a DataFrame from a GenePattern File

Our next step is to load the GenePattern file into a pandas DataFrame. Thankfully, the GenePattern Python library provides a method already configured to import any GCT (or ODF) file. Just call *gp.data.GCT()* and pass in the GenePattern file.

For loading data in a different file format, the pandas library provides the *read_csv()* method. This method accepts a file-like object or URL and attempts to import the data into a DataFrame, however, additional configuration is usually required. For example, you might need to specify the separator (tabs or commas), the number of header rows or whether to ignore white space. For more information see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).

In [ ]:
df = gp.data.GCT(gpfile.uri)

### Displaying a DataFrame

Your GenePattern file has now been loaded into a DataFrame. From here we're going to cover how to explore this data, querying it to ask meaningful biological questions. We will also provide a variety of useful tips and tricks.

The first trick is to get the DataFrame to display a preview of the encapsulated data. This will verify that all of the steps up to this point have worked, and helps you understand the data that you will be exploring. To do this, you can simply type the variable name of the DataFrame (*df*) in a cell and execute it. We've already entered it for you below.

In [ ]:
df

## Tips & Tricks

Explaining the DataFrame type's capabilities in the abstract is fine, but sometimes it can be difficult to see how to apply those capabilities to real world bioinformatic problems. Below perform some common tasks with our data.

### Selecting Genes and Samples

Sometimes you might want to examine the gene expression values for a particular gene or sample. Extacting this information from a DataFrame is easy using indexs on the the *loc* property for rows (genes) or directly for columns (samples). Just call the property, passing in the label for the gene or sample the same way you would pass in the key to a dictionary.

*Side Note:* In this dataset the genes are labeled with their Affymetrix Probe names rather than HUGO gene symbols. Were this dataset to use to use HUGO, you would use those names to select genes instead.

In [ ]:
# Select a particular sample
df['AML SH 18']

In [ ]:
# Select a particular gene
df.loc['AFFX-BioB-5_at']

### Transpose a DataFrame

Although it is standard in the GCT format (and some other gene expression formats) to have genes as the rows and samples as the columns, there may be other methods or formats that expect the opposite, essentially a transposed version of the matrix. 

This transposition is easy to achieve. Just return the DataFrame's *T* (transposition) property.

In [ ]:
df.T

### Normalize Expression Values

It's fairly common for data to benefit from normalization before using certain analytical methods. There are also many types of normalization, and covering in what circumstances you might want to use each is beyond the scope of this tutorial. 

However, below we present a quick and dirty way to perform z-score normalization on each of the rows. This same approach can be adapted when performing other tyes of normalization.

*Side Note:* In the code below, *axis=0* means to normalize over the rows, whereas axis=1 would be to normalize over the columns.

In [ ]:
# z-score normalization
(df - df.mean(axis=0))/df.std(axis=0)

### Filtering by Values

DataFrames make it easy to filter values, selecting only the rows or columns that meet certain criteria. This is a common preprocessing step, as often only a small subset of genes will be significant.

By way of example, imagine we want to select only those genes that were upregulated in the sample *ALL 19769 TA+ Norel*. This can be achieved as shown below. In this case we select the column *ALL 19769 TA+ Norel*, and for each value in the column, check to see if the number is positive.

In [ ]:
df.loc[df['ALL 19769 TA+ Norel'] > 0]

### Fold Change Filter

For a slightly more complicated filtering example, imagine we want to filter for only those genes that differ by a significant amount. To do this we can apply a *Fold Change* filter. This selects only genes whose maximum value across all samples, divided by their minimum value across all samples, is at least a specificed ratio.

$$Fold\:Change = {Max\:Across\:Samples \over Min\:Across\:Samples}$$

Below we select genes with a fold change > 3.

In [ ]:
# Fold Change Filter
df.loc[abs(df.max(axis=1) / df.min(axis=1)) > 3]

### Centering Values at Zero

At times it might be useful to center the values in a DataFrame on zero, thus calculating how much each expression value differs from the mean. This can be achieved as shown below.

In [ ]:
df.sub(df.mean(axis=1), axis=0)

## Writing a DataFrame Back to GCT

In addition to reading a GenePattern file into a DataFrame, the reverse is also possible: writing a pandas DataFrame back to disk, thereby creating a new file which GenePattern can use in its analyses. For GCT (and ODF) files, the GenePattern Python library provides methods for doing this easily, automatically handling the file format specifics.

You can write the DataFrame to a GCT file by calling the *write_gct()* function. This will create the file in the same directory as this tutorial notebook. From there you can upload the file to a GenePattern server, download the file to your local machine or with it perform any other related task.

In [ ]:
gp.data.write_gct(df, file_path='./all_aml_test.gct')

## Further Reading

This concludes the *Using GenePattern and Pandas* tutorial. For more information on GenePattern or the pandas library, please see the links below.

* More on the capabilities of GenePattern is available at the [GenePattern Notebook](http://genepattern-notebook.org/) website.
* For a more technical and comprehensive look at the pandas library we recommend the [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/Index.ipynb).